In [36]:
import pandas as pd
import numpy as np

In [37]:
bus_groups_replaceinfo = pd.read_csv('../data/bus_groups_replaceinfo.csv')

bus_groups_replaceinfo['purchase_date'] = pd.to_datetime(bus_groups_replaceinfo['purchase_date'], format='%Y-%m')
bus_groups_replaceinfo['1st_rep_date'] = pd.to_datetime(bus_groups_replaceinfo['1st_rep_date'], format='%Y-%m')
bus_groups_replaceinfo['2and_rep_date'] = pd.to_datetime(bus_groups_replaceinfo['2and_rep_date'], format='%Y-%m')
bus_groups_replaceinfo['data_begins_date'] = pd.to_datetime(bus_groups_replaceinfo['data_begins_date'], format='%Y-%m')

bus_groups_replaceinfo

,bus_name,group,purchase_date,1st_rep_date,odometer_1st,2and_rep_date,odometer_2and,data_begins_date
0,4403,1,1983-05-01,NaT,NaN,NaT,NaN,1983-05-01
1,4404,1,1983-05-01,NaT,NaN,NaT,NaN,1983-05-01
2,4405,1,1983-05-01,NaT,NaN,NaT,NaN,1983-05-01
3,4406,1,1983-05-01,NaT,NaN,NaT,NaN,1983-05-01
4,4407,1,1983-05-01,NaT,NaN,NaT,NaN,1983-05-01
...,...,...,...,...,...,...,...,...
157,4252,8,1972-02-01,1976-05-01,190700.0,NaT,NaN,1974-12-01
158,4253,8,1972-02-01,1977-01-01,151500.0,1984-09-01,334400.0,1974-12-01
159,4254,8,1972-02-01,1976-03-01,168400.0,NaT,NaN,1974-12-01
160,4255,8,1972-02-01,1976-01-01,132000.0,NaT,NaN,1974-12-01


In [38]:
bus_groups_replaceinfo_summary = (
    bus_groups_replaceinfo[["group", "odometer_1st"]]
    .groupby("group")
    .agg(["max", "min", "mean", "std"])
    .round(0)
)
bus_groups_replaceinfo_summary

odometer_1st                             
               max       min      mean      std
group                                          
1              NaN       NaN       NaN      NaN
2              NaN       NaN       NaN      NaN
3         273400.0  124800.0  199733.0  37459.0
4         387300.0  121300.0  260000.0  64683.0
5         322500.0  118000.0  245291.0  60258.0
6         237200.0   82400.0  150786.0  61007.0
7         331800.0  170800.0  230317.0  40276.0
8         297500.0  132000.0  186911.0  45221.0

In [39]:
def calculate_months_diff(date1: pd.Series, date2: pd.Series) -> pd.Series:
    result = np.where(
        date1.isna() | date2.isna(),
        np.nan,
        (date2.dt.year - date1.dt.year) * 12 + (date2.dt.month - date1.dt.month) + 1,
    )
    return pd.Series(result)

In [40]:
bus_groups_replaceinfo["1st_rep_period"] = calculate_months_diff(
    bus_groups_replaceinfo["purchase_date"], bus_groups_replaceinfo["1st_rep_date"]
)
bus_groups_replaceinfo["2and_rep_period"] = calculate_months_diff(
    bus_groups_replaceinfo["1st_rep_date"], bus_groups_replaceinfo["2and_rep_date"]
)

bus_groups_replace_period = (
    pd.concat(
        [
            bus_groups_replaceinfo[["group", "1st_rep_period"]].rename(
                columns={"1st_rep_period": "period"}
            ),
            bus_groups_replaceinfo[["group", "2and_rep_period"]].rename(
                columns={"2and_rep_period": "period"}
            ),
        ],
        axis=0,
    )
    .dropna()
    .reset_index(drop=True)
)


bus_groups_replace_period_summary = (
    bus_groups_replace_period.groupby("group")
    .agg(["max", "min", "mean", "std"])
    .round(1)
)
bus_groups_replace_period_summary

period                  
         max   min  mean   std
group                         
3       74.0  38.0  59.1  10.9
4      116.0  28.0  73.7  23.3
5      127.0  31.0  85.4  29.7
6      127.0  49.0  74.7  35.2
7      104.0  41.0  68.3  16.9
8      104.0  36.0  58.4  22.2

In [41]:
pd.merge(
    bus_groups_replaceinfo_summary,
    bus_groups_replace_period_summary,
    left_index=True,
    right_index=True,
)

odometer_1st                              period                  
               max       min      mean      std    max   min  mean   std
group                                                                   
3         273400.0  124800.0  199733.0  37459.0   74.0  38.0  59.1  10.9
4         387300.0  121300.0  260000.0  64683.0  116.0  28.0  73.7  23.3
5         322500.0  118000.0  245291.0  60258.0  127.0  31.0  85.4  29.7
6         237200.0   82400.0  150786.0  61007.0  127.0  49.0  74.7  35.2
7         331800.0  170800.0  230317.0  40276.0  104.0  41.0  68.3  16.9
8         297500.0  132000.0  186911.0  45221.0  104.0  36.0  58.4  22.2